# Assignment 3: Informed Search

##  Grid Navigation
In this section, you will investigate the problem of navigation on a two dimensional grid with obstacles. The goal is to produce the shortest path between a provided pair of points, taking care to maneuver around the obstacles as needed. Path length is measured in Euclidean distance. Valid directions of movement include up, down, left, right, up-left, up-right, down-left, and down-right. 

### Note:
You are expected to write code where you see **your code here**.  
Make sure you delete the lines with **pass** and **raise NotImplementedError** or your code may not run correctly.

### Task
Your task is to write a function find_path(start, goal, scene) which returns the shortest path from the start point to the goal point that avoids traveling through the obstacles in the grid. 

### Structure/Representation
For this problem, points will be represented as two-element tuples of the form (row, column), and scenes will be represented as two-dimensional lists of Boolean values, with False values corresponding empty spaces and True values corresponding to obstacles. 

### Output
Your output should be the list of points in the path, and should explicitly include both the start point and the goal point. If multiple optimal solutions exist, any of them may be returned. If no optimal solutions exist, or if the start point or goal point lies on an obstacle, you should return the sentinal value None. If start and goal state are the same, return None.

### Implementation
Your implementation should consist of 
* an A* search 
* the straight-line Euclidean distance heuristic. 

Helper functions are allowed and encouraged.

## 1. Euclidean distance
First, let's define a function used for euclidean distance.

In [1]:
############################################################
# Section 1 Grid Navigation - Euclidiean distance
############################################################
import math

def grid_euclidean_distance(current_state, goal_state):
    """
    Calculate the Euclidean distance between two points in a grid.
    
    :param current_state: Tuple (row, column) representing the current position.
    :param goal_state: Tuple (row, column) representing the goal position.
    :return: Euclidean distance between current_state and goal_state.
    """
    return math.sqrt((current_state[0] - goal_state[0])**2 + (current_state[1] - goal_state[1])**2)



In [2]:
##########################
### TEST YOUR SOLUTION ###
##########################

current_state1, goal_state1 = (1,1), (1,1)
assert grid_euclidean_distance(current_state1, goal_state1) == 0

current_state2, goal_state2 = (1,2), (1,1)
assert grid_euclidean_distance(current_state2, goal_state2) == 1

current_state3, goal_state3 = (3,2), (1,2)
assert grid_euclidean_distance(current_state3, goal_state3) == 2

current_state4, goal_state4 = (1,1), (2,2)
assert grid_euclidean_distance(current_state4, goal_state4) == 2**0.5
print("test passed!")

test passed!


## 2. Helper functions
Next, let's define a functions that finds the successors.

In [3]:
def grid_successors(current, scene):
    """
    Find all possible successors for a given position on the grid.
    
    :param current: Tuple (row, column) representing the current position.
    :param scene: 2D list representing the grid, with False for open spaces and True for obstacles.
    :return: Tuple of tuples representing all valid successor positions.
    """
    rows, cols = len(scene), len(scene[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    successors = []
    
    for direction in directions:
        new_row, new_col = current[0] + direction[0], current[1] + direction[1]
        
        if 0 <= new_row < rows and 0 <= new_col < cols and not scene[new_row][new_col]:
            successors.append((new_row, new_col))
    
    return tuple(successors)



In [4]:
# Test cases
scene1 = [
    [True, True, True],
    [False, False, True]
]

print(grid_successors((1, 2), scene1))  # Expecting: ((1, 1),)
print(grid_successors((0, 1), scene1))  # Expecting: ((1, 1), (1, 0))

assert grid_successors((1, 2), scene1) == ((1, 1),)
assert grid_successors((0, 1), scene1) == ((1, 1), (1, 0))
print("test passed!")

((1, 1),)
((1, 1), (1, 0))
test passed!


In [5]:
scene1 = [[True, True, True],
         [False, False, True]]
assert grid_successors((1, 2), scene1) == ((1, 1),)
assert grid_successors((0, 1), scene1) == ((1, 1),(1, 0),)
print("test passed!")

test passed!


## 3. Find path
Finally let's implement the path search.

In [6]:
############################################################
# Grid Navigation
############################################################
import collections, itertools, queue, random, copy, heapq, math

 
def find_path(start, goal, scene):
    """
    Find the shortest path from start to goal avoiding obstacles in the grid.
    
    :param start: Tuple (row, column) representing the start position.
    :param goal: Tuple (row, column) representing the goal position.
    :param scene: 2D list representing the grid, with False for open spaces and True for obstacles.
    :return: List of tuples representing the path from start to goal, or None if no path exists.
    """
    if scene[start[0]][start[1]] or scene[goal[0]][goal[1]] or start == goal:
        return None

    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    f_score = {start: grid_euclidean_distance(start, goal)}

    while open_set:
        current = heapq.heappop(open_set)[1]

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor in grid_successors(current, scene):
            tentative_g_score = g_score[current] + grid_euclidean_distance(current, neighbor)

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + grid_euclidean_distance(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None


In [7]:
##########################
### TEST YOUR SOLUTION ###
##########################
scene1 = [[False, False, False], 
          [False, True , False], 
          [False, False, False]] 

assert find_path((0, 0), (2, 1), scene1) == [(0, 0), (1, 0), (2, 1)] 

scene2 = [[False, True, False], 
          [False, True, False], 
          [False, True, False]] 
assert find_path((0, 0), (0, 2), scene2) is None
print("test passed!")

test passed!
